<a href="https://colab.research.google.com/github/Cezari0o/PLN-Trab/blob/main/a3/Atividade3_PLN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Atividade 3
Feito por Gabriel Cesário Silva Martins - 180100912

## Recuperando o corpus de notícias

In [1]:
import nltk
import pandas as pd

nltk.download('reuters')
from nltk.corpus import reuters

cats = reuters.categories()
print("Reuters has %d categories:\n%s" % (len(cats), cats))

fileids = reuters.fileids()

[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data]   Package reuters is already up-to-date!


Reuters has 90 categories:
['acq', 'alum', 'barley', 'bop', 'carcass', 'castor-oil', 'cocoa', 'coconut', 'coconut-oil', 'coffee', 'copper', 'copra-cake', 'corn', 'cotton', 'cotton-oil', 'cpi', 'cpu', 'crude', 'dfl', 'dlr', 'dmk', 'earn', 'fuel', 'gas', 'gnp', 'gold', 'grain', 'groundnut', 'groundnut-oil', 'heat', 'hog', 'housing', 'income', 'instal-debt', 'interest', 'ipi', 'iron-steel', 'jet', 'jobs', 'l-cattle', 'lead', 'lei', 'lin-oil', 'livestock', 'lumber', 'meal-feed', 'money-fx', 'money-supply', 'naphtha', 'nat-gas', 'nickel', 'nkr', 'nzdlr', 'oat', 'oilseed', 'orange', 'palladium', 'palm-oil', 'palmkernel', 'pet-chem', 'platinum', 'potato', 'propane', 'rand', 'rape-oil', 'rapeseed', 'reserves', 'retail', 'rice', 'rubber', 'rye', 'ship', 'silver', 'sorghum', 'soy-meal', 'soy-oil', 'soybean', 'strategic-metal', 'sugar', 'sun-meal', 'sun-oil', 'sunseed', 'tea', 'tin', 'trade', 'veg-oil', 'wheat', 'wpi', 'yen', 'zinc']


In [2]:
categories = []
text = []

for file in fileids:
    categories.append(reuters.categories(file))
    text.append(reuters.raw(file))

df = pd.DataFrame({'ids':fileids, 'categories':categories, 'text':text})

In [3]:
df

,ids,categories,text
0,test/14826,[trade],ASIAN EXPORTERS FEAR DAMAGE FROM U.S.-JAPAN RI...
1,test/14828,[grain],CHINA DAILY SAYS VERMIN EAT 7-12 PCT GRAIN STO...
2,test/14829,"[crude, nat-gas]",JAPAN TO REVISE LONG-TERM ENERGY DEMAND DOWNWA...
3,test/14832,"[corn, grain, rice, rubber, sugar, tin, trade]",THAI TRADE DEFICIT WIDENS IN FIRST QUARTER\n ...
4,test/14833,"[palm-oil, veg-oil]",INDONESIA SEES CPO PRICE RISING SHARPLY\n Ind...
...,...,...,...
10783,training/999,"[interest, money-fx]",U.K. MONEY MARKET SHORTAGE FORECAST REVISED DO...
10784,training/9992,[earn],KNIGHT-RIDDER INC &lt;KRN> SETS QUARTERLY\n Q...
10785,training/9993,[earn],TECHNITROL INC &lt;TNL> SETS QUARTERLY\n Qtly...
10786,training/9994,[earn],NATIONWIDE CELLULAR SERVICE INC &lt;NCEL> 4TH ...


## Funções de Suporte

In [4]:
import re

def preprocessing(text):
    t = re.sub(r'#','',text)
    t = t.lower()
    t = re.sub(r'[\,\.\'\$\!/"\(\)\-<>]+',' ',t)
    t = re.sub(r'\\u2019', '\'', t)
    t = re.sub(r'\\u002c', ',', t)
    t = re.sub(r'&lt;', ' ', t)
    t = re.sub(r':\)','happy', t)
    
    return t

In [5]:
def calc_prod_interno(b1: dict, b2: dict):
  prod_interno = 0

  for w in b1.keys():
    if w in b2:
      prod_interno += b1[w] * b2[w]

  return prod_interno

In [6]:
def calc_norma(v: dict):
  norma = 0

  for n in v:
    norma += v[n]**2
  norma = norma**0.5

  return norma

In [7]:
def cos_dissimilarity(v1: dict, v2: dict):

  pi = calc_prod_interno(v1, v2)
  
  return pi / (calc_norma(v1)*calc_norma(v2))

## Preparando a execução

### Criando bag of words

E também o vocabulário do corpus.

In [8]:
vocab = set()

for txt in df['text']:
    txt = preprocessing(txt)

    for term in txt.split():
        vocab.add(term)

vocab = sorted(vocab)
print(len(vocab))

31150


Bag of words ponderado

In [9]:
bow_pond = {}
quant_words = {}

for idx, linha in df.iterrows():

  frase = linha['text']
  doc = linha['ids']
  bow_pond[doc] = dict()

  frase = preprocessing(frase)
  quant_words[doc] = len(frase.split())

  for w in frase.split():
    if w in bow_pond[doc]:
      bow_pond[doc][w] += 1
    else:
      bow_pond[doc][w] = 1

for doc in bow_pond:
  for w in bow_pond[doc]:
    bow_pond[doc][w] /= quant_words[doc]


Bag of words de contagem

In [10]:
bow_cont = {}

for idx, linha in df.iterrows():

  frase = linha['text']
  doc = linha['ids']
  bow_cont[doc] = dict()

  frase = preprocessing(frase)
  for w in frase.split():
    if w in bow_cont[doc]:
      bow_cont[doc][w] += 1
    else:
      bow_cont[doc][w] = 1

Índice de frequência

In [ ]:
idf = {}

for word in vocab:
    idf[word] = 0
    for doc in bow_cont:
        if word in bow_cont[doc]:
            idf[word] += 1

from math import log

for w in idf:
  idf[w] = log(len(df)/idf[w])

Obtendo o dicionario tf-idf

In [ ]:
bow_tfidf = {}

for doc in bow_pond:
  bow_tfidf[doc] = {}
  for w in bow_pond[doc]:
    bow_tfidf[doc][w] = bow_pond[doc][w] * idf[w]

# bow_tfidf

Bag of words binário

In [ ]:
bow_bin = dict()

i = 0
for idx, linha in df.iterrows():

  frase = linha['text']
  doc = linha['ids']
  i += 1
  bow_bin[doc] = dict()

  frase = preprocessing(frase)
  for w in frase.split():
    bow_bin[doc][w] = 1

# bow_bin

### Ranking de similaridade

Criando o ranking para cada notícia.

A função abaixo recebe um bag of words e um limite de documentos. Basicamente, a função fará o calculo da similaridade cosseno de um texto para todos os outros no dataset, assim fazendo o ranqueamento dos mais similares. 

In [14]:
def get_rank(bow, lim = None):
  similarity_rank = dict()

  # Alocando memoria para o ranque de cada docmento
  for idx, doc in enumerate(bow):
    if idx >= lim:
      break
    similarity_rank[doc] = 10 * [(None, 0)]

  for entry_count, doc in enumerate(bow):
    if lim and entry_count >= lim:
      break

    for inner_count, doc_query in enumerate(bow):
      if lim and inner_count >= lim:
        break

      if doc != doc_query:
        similarity = cos_dissimilarity(bow[doc], bow[doc_query])      

        # Ranqueando o documento 
        rank = 10
        for id, sim_val in reversed(similarity_rank[doc]):
          if similarity > sim_val:
            rank -= 1
          else:
            break

        if rank < 10:
          similarity_rank[doc] = similarity_rank[doc][:rank] + [(doc_query, similarity)] + similarity_rank[doc][rank:-1]

  return similarity_rank

Nas chamadas abaixo, devido ao tempo excessivo de cálculo na função, foram considerados apenas metade dos 10778 documentos do corpus.

In [15]:
# Os primeiros n documentos sao considerados
n = len(df) // 2
n

5394

In [16]:
rank_tfidf = get_rank(bow_tfidf, n)

In [ ]:
rank_tfidf

Ranque para o bag of words binário

In [20]:
rank_bow_bin = get_rank(bow_bin, n)

In [ ]:
rank_bow_bin

## Ranking de Acurácia
Calculando o ranking de acurácia

In [22]:
# Função que calcula as notícias mais similares (pela categoria) dado um ranque
def get_accuracy(rank_similarity: dict):
  
  top10 = dict()
  for i in range(0, 10):
    top10[i] = 0

  for doc in rank_similarity:
    idx_doc = df.index[df['ids'] == doc][0]

    for idx, d in enumerate(rank_similarity[doc]): # d == tupla -> (documento, similaridade)
      idx_d = df.index[df['ids'] == d[0]][0]

      for cat in df.at[idx_d, 'categories']:
        if cat in df.at[idx_doc, 'categories']:
          top10[idx] += 1
          break

  for idx in top10:
    top10[idx] /= len(rank_similarity)

  return top10

Nas chamadas abaixo, temos a relação da porcentagem de notícias com a mesma categoria, dado o ranque de similaridade de cada uma.

### Ranking BoW

In [23]:
top10_bow = get_accuracy(rank_bow_bin)
top10_bow

{0: 0.8692992213570634,
 1: 0.8303670745272526,
 2: 0.7995921394141638,
 3: 0.7944011865035224,
 4: 0.7793845012977382,
 5: 0.7654801631442343,
 6: 0.753429736744531,
 7: 0.7519466073414905,
 8: 0.7450871338524286,
 9: 0.7293288839451242}

### Ranking TF-IDF

In [24]:
top10_tfidf = get_accuracy(rank_tfidf)
top10_tfidf

{0: 0.8481646273637374,
 1: 0.7977382276603634,
 2: 0.7908787541713015,
 3: 0.7860585836114201,
 4: 0.78568780126066,
 5: 0.7671486837226548,
 6: 0.7636262513904338,
 7: 0.7525027808676307,
 8: 0.7443455691509084,
 9: 0.7458286985539488}